# Train Stations Notebook

We want to find the minimal amount of transfers between two MTA stations which we can eventually reapply into an app n the future. We can do this via: http://web.mta.info/developers/data/nyct/subway/Stations.csv

The main parts of the csv file we may need to use are the station ID, division, stop name and daytime routes. We need to create a graph which connects the subway stations together so we can use breadth-first search to find the shortest route between the two routes.

In [1]:
import pandas as pd
from collections import deque, defaultdict
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = 50, 50
%matplotlib inline

In [2]:
dataset = pd.read_csv('http://web.mta.info/developers/data/nyct/subway/Stations.csv')

In [3]:
dq

NameError: name 'dq' is not defined

In [ ]:
dataset.head(5)

In [ ]:
for index, row in dataset.iterrows():
    if row['Line'] == 'Franklin Ave Shuttle':
        dataset.at[index, "Daytime Routes"] = 'Fr_S'
    elif row['Line'] == 'Lexington - Shuttle':
        dataset.at[index, 'Daytime Routes'] = 'Lex_S'

dataset = dataset.iloc[:len(dataset)-1]

In [ ]:
kstops = dataset.drop(['Complex ID', 'Division', 'GTFS Stop ID', 'Borough', 'Structure'], axis=1)
kstops.rename(index=str,columns={"GTFS Latitude" : "Lat", 'GTFS Longitude': "Long", "Station ID": "id"}, inplace=True)
kstops.head(20)

In [ ]:
trains_stops = kstops[['Stop Name', 'Daytime Routes']]
trains_stops

In [ ]:
# Split the trains individually and gather them
nyc_trains = []

trains = trains_stops['Daytime Routes']
for many_trains in trains: # For the N W in the columns
    train_list = many_trains.split(" ") # Split the N W into ['N', 'W']
    for train in train_list: # For each char in ['N', 'W']
        nyc_trains.append(train) # add it to the list

nyc_trains = sorted(list(set(nyc_trains)))
print(nyc_trains)

In [ ]:
for index, row in trains_stops.iterrows():
    print(index, row['Stop Name'], row['Daytime Routes'])

In [ ]:
# Gather all the stops made by a single train
train_to_stops = defaultdict(list)
for train in nyc_trains:
    for index, row in trains_stops.iterrows():
        if train in row['Daytime Routes'].split(" "):
            train_to_stops[train] += [row['Stop Name']]
train_to_stops

In [ ]:
stops = trains_stops['Stop Name']
stops_to_transfers = defaultdict(list)

for index, row in trains_stops.iterrows():
    stops_to_transfers[stops[index]] += trains_stops.loc[index, 'Daytime Routes'].split(" ")

stops_to_transfers

In [ ]:
def bfs(goal: str, start: str, dataset: dict):
    dq = deque()
    dq += dataset[start]
    visited = []
    while goal not in visited:
        item = dq.popleft()
        if item not in visited:
            print(item)
            visited.append(item)
    return visited

In [ ]:
bfs('Pelham Pkwy', 'Hewes St', stops_to_transfers)

In [ ]:
plt.scatter(kstops['Lat'], kstops['Long'])